# Facility Location Model

Sabi Horvat, March 2021

## Overview

The model chooses N facilities from a set of available facility locations to minimize total cost and allocates products to customers from those facilities.


## Model Formulation 

### Decision Variables

- $\text{open_DC}_{d} \in [0,1]$: Whether the Distribution Center [ d ] is opened, or whether space within an existing Distribution Center is contracted.  Either way, there is a fixed cost to the decision.   

- $\text{allocation}_{d,c} \in \mathbb{N}_{0}$: The non-negative amount of units that are planned to flow from Distribution Center [ d ] to Customer [ c ]

### Objective Function

- **Minimize the planned cost**.  Planned cost includes the fixed cost of setup and the variable cost of shipping products from the Distribution Center to Customers.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(d,c) \in \text{Distribution Centers} \times \text{Customers}}({\text{shipping_cost}_{d,c}} * {\text{allocation}_{d,c}}) +  ({\text{fixed_cost}_{d}} * {\text{open_DC}_{d}})
\end{equation}

### Constraints

- **Fulfill Customer Demand**: Customers to receive allocation from the distribution centers in the amount of the planned demand.

\begin{equation}
\sum_{d \in \text{Distribution Centers}}{\text{allocation}_{d,c}} \geq \text{Demand}_{c} \quad  \forall c \in \text{Customers}
\end{equation}

- **Only allocate products to Open Distribution Centers**: A modeling constraint to ensure closed distribution centers do not ship products.  The big M is set to the sum of all customer demand.

\begin{equation}
\ {\text{allocation}_{d,c}} \leq M * \text{open_DC}_{d} \quad  \forall c \in \text{Customers, } d \in \text{Distributions}
\end{equation}

- **Distribution Centers Capacity**: Each Distribution Center has a limited capacity based on these planning decisions.

\begin{equation}
\sum_{c \in \text{Customers}}{\text{allocation}_{d,c}} \leq \text{Capacity}_{d} \quad \forall d \in \text{Distribution Centers}
\end{equation}
 

In [1]:
import numpy as np  
import pandas as pd 
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

from itertools import product
from pulp import (LpProblem, LpMinimize, LpVariable, lpSum, 
                  PULP_CBC_CMD, GLPK_CMD, LpStatus, value) 

In [22]:
distribution_center = pd.DataFrame.from_dict({  
    'PortlandME' : ['PortlandME', 43.7, -70.3, (-70.3, 43.7), 100, 200],
    'PortlandOR' : ['PortlandOR', 45.5, -122.6, (-122.6, 45.5), 100, 200],
    'ChicagoIL' : ['ChicagoIL', 41.8, -87.7, (-87.7, 41.8), 500, 200],
    'DallasTX' : ['DallasTX', 32.8, -96.8, (-96.8, 32.8), 100, 200],
    'MiamiFL' : ['MiamiFL', 25.8, -80.2, (-80.2, 25.8), 100, 200],
    'SanDiegoCA' : ['SanDiegoCA', 32.9, -117.1, (-117.1, 32.9), 100, 200],
    'SeattleWA' : ['SeattleWA', 47.6, -122.3, (-122.3, 47.6), 100, 200]},
    orient = 'index',
    columns = ["Distribution_Center", 'Latitude', 'Longitude', 'LongLat', 'FixedCost', 'Capacity'])
distribution_center = distribution_center.reset_index()
distribution_center['index_join'] = distribution_center.index
n_distribution_center = len(distribution_center)
distribution_center[['Distribution_Center','FixedCost','Capacity']]

,Distribution_Center,FixedCost,Capacity
0,PortlandME,100,200
1,PortlandOR,100,200
2,ChicagoIL,500,200
3,DallasTX,100,200
4,MiamiFL,100,200
5,SanDiegoCA,100,200
6,SeattleWA,100,200


In [23]:
customer = pd.DataFrame.from_dict({ 'Customer1': ['Customer1', 50, -90, (-90, 50), 100],
                                    'Customer2': ['Customer2', 40, -100, (-100, 40), 200],
                                    'Customer3': ['Customer3', 30, -85, (-70, 30), 300]},
                                    orient = 'index',
                                    columns = ['Customer', 'Latitude', 'Longitude', 'LongLat', 'Demand'])
customer = customer.reset_index()
n_customer = len(customer)
customer[['Customer','Demand']]

,Customer,Demand
0,Customer1,100
1,Customer2,200
2,Customer3,300


In [4]:
# set of all possible ordered pairs, also known as the cartesian product 
routes = list(product(range(n_distribution_center), range(n_customer)))
all_customers = list(range(n_customer))
open_distribution_centers = list(range(n_distribution_center)) 
fixed_cost = {(d): distribution_center.FixedCost[d] for d in open_distribution_centers}
variable_names = [str(i)+'__'+str(j) for i in range(0, n_distribution_center) for j in range(0, n_customer)]
#variable_names = [str(i)+'_'+str(j) for i in range(1, n_distribution_center+1) for j in range(1, n_customer+1)]


In [5]:
# Euclidean distance, i.e. straight line distance rather than driving routes
cost_per_distance = 1
def euclidean_distance(loc1, loc2):
    """
    Calculates the straightline distance between two points.
    
    This is an approximation for this model and:
    * does not calculate actual miles or kilometers from latitude and longitude
    * does not use rectilinear distance or driving distance that may be more real
    * does not use the haversine formula to calculate distance based on Earth curvature
    
    """
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return pow(dx*dx + dy*dy,0.5)

shipping_cost = {(i, j): cost_per_distance*euclidean_distance(distribution_center.LongLat[i], 
                                            customer.LongLat[j]) for i, j in routes}

## Model


In [6]:
# PuLP model formulation using Pandas
# create model to minimize cost
model = LpProblem("Facility_Location_Problem", LpMinimize) 

# decision variables
decision_variable_x = LpVariable.matrix("Allocation", variable_names, cat = 'Integer', lowBound=0) 
open_DC = LpVariable.matrix("Open_Distribution_Center", open_distribution_centers, cat = 'Binary') 
allocation = np.array(decision_variable_x).reshape(7,3) # since indices must be integer for PuLP

# The objective function minimizes the cost
model += lpSum(shipping_cost[(d, c)]*allocation[d][c] for d, c in routes) + \
         lpSum(fixed_cost[d]*open_DC[d] for d in open_distribution_centers)

# Constraint: Each customer demand should be fulfillled
for c in customer.index:
    model += lpSum(allocation[d][c] for d in open_distribution_centers) >= customer.Demand[c]
    
# Constraint: Only allocate product to assigned (i.e. opened/built/active) distribution centers
my_M = sum(customer.Demand) # sufficiently large M to enforce constraints
for d, c in routes:
    model += allocation[d][c] <= my_M*open_DC[d] # (1)

# Constraint: Capacity at each DC cannot be exceeded
for d in open_distribution_centers:
    model += lpSum(allocation[d][c] for c in all_customers) <= distribution_center.Capacity[d]


In [7]:
# Solve the model
model.solve()

# The status of the solution (Optimal, Infeasible, Unbounded, Not Solved, or Undefined)
print("Status:", LpStatus[model.status])

# The objective solution
# for v in model.variables():
#     print(v.name, "=", v.varValue)

# The optimal objective function value 
print("\nTotal Cost = ", value(model.objective))

Status: Optimal

Total Cost =  6803.96429951784


In [8]:
# Objective Solution

result_value = []
for v in model.variables():
    result_value.append(v.varValue)

result_route = []
for v in model.variables():
    result_route.append(v.name)

result_od = pd.DataFrame(result_route, columns=['result_string'])
result_od[['variable','DC_Cust']] = result_od['result_string'].str.split('_',1, expand=True)
result_od[['Distribution_Center_Num','Customer_Num']] = result_od['DC_Cust'].str.split('__',1, expand=True)
result_od = result_od.drop(['result_string','DC_Cust','variable'],axis=1)
result_od['flow'] = result_value
result_od = result_od[result_od['flow'] > 0].dropna(axis = 0) 
result_od.Distribution_Center_Num = result_od.Distribution_Center_Num.astype('int64')
# add cost to the solution as well (i.e. flow * distance = cost)

In [9]:
result_od

,Distribution_Center_Num,Customer_Num,flow
2,0,2,100.0
6,2,0,100.0
10,3,1,200.0
14,4,2,200.0


In [10]:
temp = result_od.merge(distribution_center, left_on='Distribution_Center_Num', right_on='index_join', how = 'inner')
result_od1 = temp[['Distribution_Center','Latitude','Longitude','LongLat','Capacity','Customer_Num','flow',]].copy()
customer['join_index'] = customer.index
result_od1.Customer_Num = result_od1.Customer_Num.astype('int64')

In [11]:
result_od1

,Distribution_Center,Latitude,Longitude,LongLat,Capacity,Customer_Num,flow
0,PortlandME,43.7,-70.3,"(-70.3, 43.7)",200,2,100.0
1,ChicagoIL,41.8,-87.7,"(-87.7, 41.8)",200,0,100.0
2,DallasTX,32.8,-96.8,"(-96.8, 32.8)",200,1,200.0
3,MiamiFL,25.8,-80.2,"(-80.2, 25.8)",200,2,200.0


In [12]:
temp = result_od1.merge(customer, left_on='Customer_Num', right_on='join_index', how='inner') 
result_od2 = temp[['Distribution_Center','Latitude_x','Longitude_x','LongLat_x','Capacity','Customer','Latitude_y','Longitude_y','LongLat_y','Demand','flow',]]
result_od2

,Distribution_Center,Latitude_x,Longitude_x,LongLat_x,Capacity,Customer,Latitude_y,Longitude_y,LongLat_y,Demand,flow
0,PortlandME,43.7,-70.3,"(-70.3, 43.7)",200,Customer3,30,-85,"(-70, 30)",300,100.0
1,MiamiFL,25.8,-80.2,"(-80.2, 25.8)",200,Customer3,30,-85,"(-70, 30)",300,200.0
2,ChicagoIL,41.8,-87.7,"(-87.7, 41.8)",200,Customer1,50,-90,"(-90, 50)",100,100.0
3,DallasTX,32.8,-96.8,"(-96.8, 32.8)",200,Customer2,40,-100,"(-100, 40)",200,200.0


In [13]:
result_od2 = result_od2.assign(unit_shipping_cost = pow(\
            pow(result_od2['Longitude_x'] - result_od2['Longitude_y'], 2) + \
            pow(result_od2['Latitude_x'] - result_od2['Latitude_y'], 2), 0.5))


In [14]:
result_od2

,Distribution_Center,Latitude_x,Longitude_x,LongLat_x,Capacity,Customer,Latitude_y,Longitude_y,LongLat_y,Demand,flow,unit_shipping_cost
0,PortlandME,43.7,-70.3,"(-70.3, 43.7)",200,Customer3,30,-85,"(-70, 30)",300,100.0,20.094278
1,MiamiFL,25.8,-80.2,"(-80.2, 25.8)",200,Customer3,30,-85,"(-70, 30)",300,200.0,6.378087
2,ChicagoIL,41.8,-87.7,"(-87.7, 41.8)",200,Customer1,50,-90,"(-90, 50)",100,100.0,8.516455
3,DallasTX,32.8,-96.8,"(-96.8, 32.8)",200,Customer2,40,-100,"(-100, 40)",200,200.0,7.879086


In [15]:
# The objective solution
for v in model.variables():
    print(v.name, "=", v.varValue)

Allocation_0__0 = 0.0
Allocation_0__1 = 0.0
Allocation_0__2 = 100.0
Allocation_1__0 = 0.0
Allocation_1__1 = 0.0
Allocation_1__2 = 0.0
Allocation_2__0 = 100.0
Allocation_2__1 = 0.0
Allocation_2__2 = 0.0
Allocation_3__0 = 0.0
Allocation_3__1 = 200.0
Allocation_3__2 = 0.0
Allocation_4__0 = 0.0
Allocation_4__1 = 0.0
Allocation_4__2 = 200.0
Allocation_5__0 = 0.0
Allocation_5__1 = 0.0
Allocation_5__2 = 0.0
Allocation_6__0 = 0.0
Allocation_6__1 = 0.0
Allocation_6__2 = 0.0
Open_Distribution_Center_0 = 1.0
Open_Distribution_Center_1 = 0.0
Open_Distribution_Center_2 = 1.0
Open_Distribution_Center_3 = 1.0
Open_Distribution_Center_4 = 1.0
Open_Distribution_Center_5 = 0.0
Open_Distribution_Center_6 = 0.0


In [16]:
# create a map with folium
import folium
center = (40,-95)
m = folium.Map(location=center, width='100%', height='100%',
               zoom_start=4, tiles = 'Stamen Toner')


In [17]:
# loop on rows to create O-D pairs
add_lines_df = pd.DataFrame()
for i in range(0,len(result_od2)):
    origin = pd.DataFrame((result_od2['Latitude_x'][i],result_od2['Longitude_x'][i]))
    destination = (result_od2['Latitude_y'][i],result_od2['Longitude_y'][i])
    od = [(result_od2['Latitude_x'][i],result_od2['Longitude_x'][i]),
         (result_od2['Latitude_y'][i],result_od2['Longitude_y'][i])]
    add_lines_df[i] = od
    folium.PolyLine(add_lines_df[i], color='orange').add_to(m)

In [18]:
# large green circles for distribution centers
for i in range(len(result_od2)): folium.CircleMarker(
    location=[result_od2.iloc[i]['Latitude_x'],
              result_od2.iloc[i]['Longitude_x']],
    color = 'green',
    fill = True,
    fill_opacity = 0.7,
    radius=10,
    ).add_to(m)

In [19]:
# small blue circles for customers
for i in range(len(result_od2)): folium.CircleMarker(
    location=[result_od2.iloc[i]['Latitude_y'],
              result_od2.iloc[i]['Longitude_y']],
    color = 'blue',
    fill = True,
    fill_opacity = 0.7,
    radius=5,
    ).add_to(m)

In [20]:
# adds a legend to the map
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>
 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend </div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:blue;opacity:0.7;'></span>Customers</li>
    <li><span style='background:green;opacity:0.7;'></span>Distribution Centers</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

In [21]:
m.add_child(macro)